In [17]:
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

import os
from dotenv import load_dotenv

# Load environment variables (for API keys)
load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [18]:
import pyttsx3
import speech_recognition as sr
import time
import json

# Initialize TTS engine
engine = pyttsx3.init()
engine.setProperty('rate', 150)

recognizer = sr.Recognizer()
mic = sr.Microphone()


In [19]:


llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",)

# Define an improved feedback prompt template
# feedback_template = """
# You are an empathetic interview coach helping a candidate improve their responses to common interview questions. The candidate's answers may contain speech recognition errors or be incomplete.

# QUESTION: {question}
# CANDIDATE'S ANSWER: {answer}

# Please provide brief, actionable feedback that:
# 1. Acknowledges possible speech recognition errors without dwelling on them
# 2. Focuses on what a strong answer should contain
# 3. Provides specific, industry-relevant guidance
# 4. Is supportive and encouraging

# Your response should be concise (max 50 words) and follow this structure:
# followup quesiton is {followup_question}
# Be conversational, supportive, and specific.

# the ouput should consisint and oppoint 
# """
feedback_template = """
You are a helpful visa interview coach. Your feedback will be spoken aloud to the applicant through text-to-speech, so write in a natural speaking style without sections, bullet points, or special formatting.

QUESTION ASKED: {question}
APPLICANT'S ANSWER: {answer}
NEXT QUESTION IN QUEUE: {followup_question}

Provide brief, conversational feedback (60-80 words) that:
1. Acknowledges one strength in their answer
2. Suggests one specific improvement
3. Smoothly transitions to the next question

Guidelines:
- Use natural speech patterns suitable for speaking aloud
- Avoid phrases like "FEEDBACK:", "STRENGTHS:", or any headings
- Don't repeat the applicant's full answer back to them
- Don't use special characters, bullet points, or numbering
- End by clearly stating "Now for your next question:" followed by the exact text of the follow-up question , if there is no followup question conclude it key point to imporve 

Example of good response style:
"That's a good point about your return ticket. You might want to mention your job commitments back home as well, as this shows ties to your country. Now for your next question: What is the purpose of your visit?"
"""
feedback_prompt = PromptTemplate(
    input_variables=["question", "answer","followup_question"],
    template=feedback_template
)

chain=feedback_prompt|llm

In [ ]:

questions = [
    "Why did you choose this particular time to travel?.",
    "What is the purpose of getting vica ?",
    "Why are you traveling at this time?",
    "Have you traveled abroad before?",
    "Do you have travel insurance?",
    "What do you do for a living in your home country?"
]

In [21]:
def speak(text):
    engine.say(text)
    engine.runAndWait()


In [22]:
def listen_to_answer():
    with mic as source:
        recognizer.adjust_for_ambient_noise(source)
        print("🎙️ Listening...")
        audio = recognizer.listen(source)

    try:
        response = recognizer.recognize_google(audio)
        print(f"🗣️ You said: {response}")
        return response
    except sr.UnknownValueError:
        print("❌ Sorry, I couldn't understand you.")
        return "[Unrecognized speech]"
    except sr.RequestError:
        print("⚠️ Could not reach the speech service.")
        return "[Speech service error]"

In [23]:
responses = []
def run_mock_interview():
    print("🤖 Starting your mock interview...")
    speak("Welcome to your mock interview. Let's begin.")
    speak(questions[0])

    for i, q in enumerate(questions):
        print(f"\n🧠 Question: {q}")
        # speak(q)
        
        # Listen to the candidate's answer
        answer = listen_to_answer()
        
        # Determine the follow-up question
        next_question = questions[i + 1] if i + 1 < len(questions) else "No further questions."
        
        # Pass the question, answer, and follow-up question to the LLM
        result = chain.invoke({'question': q, "answer": answer, "followup_question": next_question})
        
        # Speak the LLM's response
        speak(result.content)
        
        # Save the response
        responses.append({
            "question": q,
            "answer": answer,
            "feedback": result.content
        })
        
        input("Press Enter for the next question...")

    # Save responses to a file
    with open("interview_responses.json", "w", encoding='utf-8') as f:
        json.dump(responses, f, indent=4)

    speak("Thank you. This concludes your mock interview.")
    print("\n✅ Done! Your responses have been saved to 'interview_responses.json'.")


In [24]:
run_mock_interview()

🤖 Starting your mock interview...

🧠 Question: Why did you choose this particular time to travel?.
🎙️ Listening...
🗣️ You said: I like to travel at this time

🧠 Question: What is the purpose of getig vica ?
🎙️ Listening...
🗣️ You said: forgetting permission into leaving another country

🧠 Question: Why are you traveling at this time?
🎙️ Listening...
🗣️ You said: what is Bittu travel

🧠 Question: Have you traveled abroad before?
🎙️ Listening...
❌ Sorry, I couldn't understand you.

🧠 Question: Do you have travel insurance?
🎙️ Listening...
❌ Sorry, I couldn't understand you.

🧠 Question: What do you do for a living in your home country?
🎙️ Listening...
❌ Sorry, I couldn't understand you.

✅ Done! Your responses have been saved to 'interview_responses.json'.
